In [ ]:
import os
import shutil
import open3d as o3d
from utils import XMLScene, parse_spec, xml_survey
import subprocess
import multiprocessing

# How to generate data XMLs using this script

1       
        a if using .ply files, place them in the helios-plusplus-folder under a folder called bim_ply or provide the path

        b make sure that the files for each building are kept in different folders named as bim_0, bim_1...
    
        c the files will be converted to .obj and placed under data/sceneparts/bim_parts with the same folder name and file name for each of the files

2   a if using .obj files place them under data/sceneparts/bim_parts. name the folder for each site as bim_0, bim_1 ...

3   the scene will be generated from these folder and placed under data/scene/bim_scenes and named as bim_id_scene.xml

4   the survey will be generated from these scenes. Platform position and scanner setting will be asked for each of the scanner
    these files would be placed under data/survey/bim_surveys and named as bim_id_survey.xml

5   using a bash file run the generated surveys using helios++ terminal command

Note:

If you want to generate multiple scenes then give the path to the folder than contains a folder for every scene's sceneparts

You can also generate a singles scene by placing only one folder inside the main folder, or by providing the path of the folder containing the sceneparts

In [ ]:
path_to_input = (input('Enter the path to the main folder containing sceneparts')) # take the path to the input files
print(f'Folder name = {os.path.basename(path_to_input)}')
#checking if there are multiple scenes
mult_scenes = 0
if len(os.listdir(path_to_input)) == 0:
     raise Exception(f'[[{path_to_input}]] is empty')
else:
     types = set()
     for file in os.listdir(path_to_input):
          if os.path.isdir(os.path.join(path_to_input,file)):
               types.add(1)
          else:
               types.add(0)
          if len(types)>1:
               raise Exception(f'[[{path_to_input}]] contains both files and folders')
     if list(types)[0] == 1:
          mult_scenes = 1

# creating a directory in data/sceneparts to save the sceneparts as .obj
parts_directory = os.path.join('data/sceneparts',os.path.basename(path_to_input).lower())
if os.path.exists(parts_directory):
     shutil.rmtree(parts_directory)
os.mkdir(parts_directory)
mesh_formats = set(['.obj','.ply','.stl','.off','.gltf'])
if mult_scenes == 0:
     for file in os.listdir(path_to_input):
          ext = os.path.splitext(file)
          if ext[1] not in mesh_formats:
               raise Exception(f'{file} not a valid mesh format')
          elif ext[1] != '.obj':
               #converting and saving to directory
               mesh_file = o3d.io.read_triangle_mesh(os.path.join(path_to_input,file))
               o3d.io.write_triangle_mesh(os.path.join(parts_directory,ext[0]+'.obj'),mesh_file)
          else:
               shutil.copy(os.path.join(path_to_input,file),os.path.join(parts_directory,ext[0]+'.obj'))
else:
     for folder in os.listdir(path_to_input):
          temp_part_dir = os.path.join(parts_directory,folder)
          if os.path.exists(temp_part_dir):
               shutil.rmtree(temp_part_dir)
          os.mkdir(temp_part_dir)
          for file in os.listdir(os.path.join(path_to_input,folder)):
               ext = os.path.splitext(file)
               if ext[1] not in mesh_formats:
                    raise Exception(f'{file} not a valid mesh format')
               elif ext[1] != '.obj':
                    #converting and saving to directory
                    mesh_file = o3d.io.read_triangle_mesh(os.path.join(os.path.join(path_to_input,folder),file))
                    o3d.io.write_triangle_mesh(os.path.join(temp_part_dir,ext[0]+'.obj'),mesh_file)
               else:
                    shutil.copy(os.path.join(os.path.join(path_to_input,folder),file),os.path.join(temp_part_dir,ext[0]+'.obj'))

print("Scene Parts Generated")
print(f"Scenes will be generated and put under [[data/scenes/{os.path.basename(path_to_input).lower()}_scenes]]")
if os.path.exists(os.path.join('data/scenes',os.path.basename(path_to_input).lower()+'_scenes')):
     shutil.rmtree(os.path.join('data/scenes',os.path.basename(path_to_input).lower()+'_scenes'))
os.mkdir(os.path.join('data/scenes',os.path.basename(path_to_input).lower()+'_scenes'))

PATH_TO_SCENES = os.path.join('data/scenes',os.path.basename(path_to_input).lower()+'_scenes')

if mult_scenes == 0:
     scene,links = XMLScene(parts_directory).result
     fin_dir = os.path.join(os.path.join('data/scenes',os.path.basename(path_to_input).lower()+'_scenes'),os.path.basename(path_to_input).lower()+'_scene')
     if os.path.exists(fin_dir):
          shutil.rmtree(fin_dir)
     os.mkdir(fin_dir)
     with open(os.path.join(fin_dir,'scene.xml'),'w',encoding='UTF-8') as f:
          f.write('<?xml version="1.0" encoding="UTF-8"?>\n'+scene)
     with open(os.path.join(fin_dir,'links.txt'),'w') as f:
          f.write(str(links))
else:
     for folder in os.listdir(parts_directory):
          temp_parts_directory = os.path.join(parts_directory,folder)
          scene,links = XMLScene(temp_parts_directory).result
          fin_dir =  os.path.join(os.path.join('data/scenes',os.path.basename(path_to_input).lower()+'_scenes'),folder.lower()+'_scene')
          if os.path.exists(fin_dir):
               shutil.rmtree(fin_dir)
          os.mkdir(fin_dir)
          with open(os.path.join(fin_dir,'scene.xml'),'w',encoding='UTF-8') as f:
               f.write('<?xml version="1.0" encoding="UTF-8"?>\n'+scene)
          with open(os.path.join(fin_dir,'links.txt'),'w') as f:
               f.write(str(links))



# For generating Survey XMLs

1. Provide with the path to the folder with the configurations for each of the sene
2. The configurations should follow the following format:
    1. the first line is the id of the scanner. each subsequent line is a new position of the scanner
    2. format:

    <scanner_id>

    <x_coord> <y_coord> <z_coord> <pulseFreq_hz> <verticalAngleMin_deg> <verticalAngleMax_deg> <scanFreq_hz> <headRotatePerSec_deg> <headRotateStart_deg> <headRotateEnd_deg> <trajectoryTimeInterval_s>

    3. example:

    <scanner_id : riegl_vz400>

    <x : 10> <y : 20> <z : 30> <verticalAngleMin_deg : 20> <pulseFreq_hz : 100000>
    
    4. Configurations specs not provided will be set to default 
    
3. Path to spec files must be the folder containing these spec files. The name of the spec files must match the name of the scenepart folder for a particular scene 

In [ ]:
path_to_spec = input()# provide path to .txt file if using in single scene mode. provide path to the folders containing the .txt files is using in multiple scene mode
if not os.path.exists(path_to_spec):
    raise Exception(f'[[{path_to_spec}]] Does not exist')

if mult_scenes != 0:
    if len(os.listdir(path_to_spec)) == 0:
        raise Exception(f'[[{path_to_spec}]] Is Empty!!')

if mult_scenes!=0:
    if not len(os.listdir(path_to_spec)) == len(os.listdir(path_to_input)):
        raise Exception(f'Mismatch in number of files.')
    if len(set([os.path.splitext(i)[1] for i in os.listdir(path_to_spec)])) != 1:
        raise Exception('\nFollowing must be in .txt format\n'.join([i for i in os.listdir(path_to_spec) if os.path.splitext(i)[1] != '.txt']))

elif mult_scenes == 0:
    if os.path.splitext(path_to_spec)[1] != '.txt':
        raise Exception(f'\nSingle Scene Mode. Path to the file must be given\nInvalid spec file extension. Please use .txt')

if mult_scenes != 0:
    input_files = [os.path.splitext(f)[0] for f in os.listdir(path_to_spec)]
    for file in os.listdir(path_to_input):
        print(os.path.splitext(file))
        if not os.path.splitext(file)[0] in input_files:
            raise Exception(f'Could not find a spec file for {file}\nCheck File names')
else:
    if os.path.splitext(path_to_input)[0] != os.path.splitext(path_to_spec)[0]:
        raise Exception(f'File names do not match {os.path.splitext(path_to_input)[0]} != { os.path.splitext(path_to_spec)[0]}')
    
print("Generating Survey XMLs")
print(f"surveys will be generated and stored under : [[/data/surveys/{os.path.splitext(path_to_spec)[0]}_surveys]]")

if os.path.exists(f'data/surveys/{os.path.splitext(os.path.basename(path_to_spec))[0]}_surveys'):
        shutil.rmtree(f'data/surveys/{os.path.splitext(os.path.basename(path_to_spec))[0]}_surveys')
os.mkdir(f'data/surveys/{os.path.splitext(os.path.basename(path_to_spec))[0]}_surveys')

SURVEYS_TO_RUN = []
if mult_scenes == 0 :
    scanner_id, parsed_specs = parse_spec(path_to_spec)
    survey_name = os.path.splitext(os.path.basename(path_to_spec))[0]+'_survey'
    scene_path  = f'data/scenes/{os.path.basename(os.path.basename(path_to_spec)).lower()}_scenes/{os.path.splitext(os.path.basename(path_to_input).lower())[0]}_scene/scene.xml#{os.path.splitext(os.path.basename(path_to_input).lower())[0]}_scene'
    survey = xml_survey((scanner_id,parsed_specs),survey_name,scene_path)
    with open(f'data/surveys/{os.path.splitext(os.path.basename(path_to_spec))[0]}_surveys/{survey_name}.xml','w') as f:
        f.write(survey)
    SURVEYS_TO_RUN.append(f'data/surveys/{os.path.splitext(os.path.basename(path_to_spec))[0]}_surveys/{survey_name}.xml')
else:
    
    for spec_file in os.listdir(path_to_spec):
        parsed_specs  = parse_spec(os.path.join(path_to_spec,spec_file))
        survey_name = os.path.splitext(os.path.basename(spec_file))[0]+'_survey'
        scene_path = os.path.join(os.path.join(os.path.join('data/scenes',os.path.basename(path_to_input).lower()+'_scenes'),os.path.splitext(spec_file)[0]+'_scene'),f'scene.xml#{os.path.splitext(spec_file)[0]}_scene')
        survey = xml_survey(parsed_specs,survey_name,scene_path)
        with open(f'data/surveys/{os.path.splitext(os.path.basename(path_to_spec))[0]}_surveys/{survey_name}.xml','w') as f:
            f.write(survey)
        SURVEYS_TO_RUN.append(f'data/surveys/{os.path.splitext(os.path.basename(path_to_spec))[0]}_surveys/{survey_name}.xml')


In [ ]:
gen_surveys = int(input("Do you want to generate surveys (1 for yes): "))
use_multi = int(input("Use multiprocessing (1 for yes): "))

if gen_surveys == 1:
    m = '\n'.join(SURVEYS_TO_RUN)
    print(f"Following surveys will be generated:\n{m}")
    if use_multi != 1 :
        for survey in SURVEYS_TO_RUN:
            print(f"Generating : {survey}")
            process = subprocess.call('run\helios '+survey)
            print(process)
